# 🤝 A2A Agent Demo — From REST API to Agent-to-Agent Protocol

## Step-by-step client demo

This notebook demonstrates how Azure API Management transforms REST APIs into **A2A (Agent-to-Agent) protocol** servers — and how new agents become **automatically discoverable** in Azure API Center.

### Demo story
1. **Deploy 2 A2A agents** (Title Generator, Outline Generator) from existing REST APIs
2. **Verify** they are registered and discoverable in API Center (kind: `rest` + kind: `a2a`)
3. **Test** agent discovery via `GET /.well-known/agent.json` (Agent Card)
4. **Test** A2A invocation via `POST /` with JSON-RPC 2.0 `message/send`
5. **Add a 3rd A2A agent** (Summary Generator) as an add-on deployment
6. **Show** it is immediately discoverable in API Center without any manual registration

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Deploy Infrastructure + 2 A2A Agents
---

### 0️⃣ Initialize

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')
import utils

deployment_name = "a2a-demo"
resource_group_name = f"rg-lab-{deployment_name}"
resource_group_location = "uksouth"

apim_sku = "Basicv2"
apim_name = "apim-a2a-demo"
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

apic_location = "uksouth"
apic_service_name_prefix = "apic-a2a"

utils.print_ok("Variables initialized")
print(f"  Resource Group: {resource_group_name}")
print(f"  APIM SKU:       {apim_sku}")
print(f"  Location:       {resource_group_location}")

### 1️⃣ Deploy infrastructure + 2 A2A agents using 🦾 Bicep

Deploys in a single Bicep template:
| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights |
| **API Gateway** | API Management (Basicv2) |
| **API Governance** | API Center (with `api`, `mcp`, and `a2a` environments) |
| **Title Agent** | REST API → A2A Server (agent card + message/send) |
| **Outline Agent** | REST API → A2A Server (agent card + message/send) |

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [ ]:
# Create the resource group
utils.create_resource_group(resource_group_name, resource_group_location)

# Build Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "apimName": { "value": apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": apic_location },
        "apicServiceNamePrefix": { "value": apic_service_name_prefix }
    }
}

with open('params-a2a-demo.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file demo-a2a-initial.bicep --parameters params-a2a-demo.json",
    f"✅ Deployment '{deployment_name}' succeeded",
    f"❌ Deployment '{deployment_name}' failed"
)

### 2️⃣ Retrieve deployment outputs

In [ ]:
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}",
                   f"Retrieved deployment outputs",
                   f"Failed to retrieve deployment")

if output.success and output.json_data:
    apim_service_name = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service Name')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apic_service_name = utils.get_deployment_output(output, 'apicServiceName', 'API Center Name')
    apic_api_env = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'API Center API Environment')
    apic_mcp_env = utils.get_deployment_output(output, 'apicMcpEnvironmentName', 'API Center MCP Environment')
    apic_a2a_env = utils.get_deployment_output(output, 'apicA2aEnvironmentName', 'API Center A2A Environment')

    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    api_key = apim_subscriptions[0].get("key")
    utils.print_info(f"API Key: ****{api_key[-4:]}")

    title_agent_endpoint = utils.get_deployment_output(output, 'titleAgentEndpoint', 'Title Agent')
    title_agent_card_endpoint = utils.get_deployment_output(output, 'titleAgentCardEndpoint', 'Title Agent Card')
    outline_agent_endpoint = utils.get_deployment_output(output, 'outlineAgentEndpoint', 'Outline Agent')
    outline_agent_card_endpoint = utils.get_deployment_output(output, 'outlineAgentCardEndpoint', 'Outline Agent Card')

---
## Part 2 — Verify Discoverability in API Center
---

### 3️⃣ List all APIs in API Center

After deployment, both the REST APIs **and** A2A agents are automatically registered in API Center.  
Notice the `kind` column — `rest` for APIs, `a2a` for A2A agents.

In [ ]:
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Count and snapshot by kind
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} -o json',
    "", "")
if output.success and output.json_data:
    apis_before = output.json_data  # snapshot for later comparison
    rest_count = sum(1 for api in apis_before if api.get('kind') == 'rest')
    a2a_count = sum(1 for api in apis_before if api.get('kind') == 'a2a')
    print(f"\n📊 Total: {len(apis_before)} APIs registered — {rest_count} REST APIs, {a2a_count} A2A Agents")

### ✅ Validate: Initial deployment — 2 REST APIs + 2 A2A Agents

Verify that exactly the expected APIs are registered before adding the Summary Agent.

In [ ]:
# Validation: check expected APIs are registered
expected_before = {
    "rest": {"title-generator-api", "outline-generator-api"},
    "a2a":  {"title-agent", "outline-agent"}
}

actual_rest = {api['name'] for api in apis_before if api.get('kind') == 'rest'}
actual_a2a  = {api['name'] for api in apis_before if api.get('kind') == 'a2a'}

print("🔍 Validation — Initial Deployment")
print("-" * 50)

# Check REST APIs
missing_rest = expected_before["rest"] - actual_rest
extra_rest = actual_rest - expected_before["rest"]
if not missing_rest:
    utils.print_ok(f"REST APIs: all {len(expected_before['rest'])} expected APIs found ✅")
else:
    utils.print_error(f"REST APIs: missing {missing_rest}")
if extra_rest:
    utils.print_info(f"  Additional REST APIs: {extra_rest}")

# Check A2A Agents
missing_a2a = expected_before["a2a"] - actual_a2a
extra_a2a = actual_a2a - expected_before["a2a"]
if not missing_a2a:
    utils.print_ok(f"A2A Agents: all {len(expected_before['a2a'])} expected agents found ✅")
else:
    utils.print_error(f"A2A Agents: missing {missing_a2a}")
if extra_a2a:
    utils.print_info(f"  Additional A2A Agents: {extra_a2a}")

# Confirm summary agent is NOT yet present
if "summary-generator-api" not in actual_rest and "summary-agent" not in actual_a2a:
    utils.print_ok("Summary Agent is NOT yet registered — ready for add-on demo ✅")
else:
    utils.print_error("⚠️ Summary Agent already exists — re-deploy without it first for a clean demo")

---
## Part 3 — Test A2A Agent Discovery & Invocation
---

### 🔍 Test Agent Discovery — GET Agent Card

The A2A protocol defines agent discovery via `GET /.well-known/agent.json`.  
This returns the **Agent Card** — a JSON document describing the agent's capabilities, skills, and supported modes.

In [ ]:
import requests, json

all_passed = True
agent_cards = {}

for agent_name, card_url in [("Title Agent", title_agent_card_endpoint),
                              ("Outline Agent", outline_agent_card_endpoint)]:
    utils.print_info(f"Discovering {agent_name} → GET {card_url}")
    response = requests.get(card_url, headers={"Accept": "application/json"})
    
    if response.status_code == 200:
        card = response.json()
        agent_cards[agent_name] = card
        utils.print_ok(f"{agent_name}: Agent Card retrieved ✅")
        print(f"  Name:        {card.get('name')}")
        print(f"  Version:     {card.get('version')}")
        print(f"  URL:         {card.get('url')}")
        skills = card.get('skills', [])
        for skill in skills:
            print(f"  Skill:       {skill.get('id')} — {skill.get('description')}")
        print()
    else:
        utils.print_error(f"{agent_name}: HTTP {response.status_code}")
        all_passed = False

if all_passed:
    utils.print_ok("🎉 All agent cards discovered successfully!")

### 🧪 Test Title Agent — A2A `message/send`

The A2A protocol uses **JSON-RPC 2.0** over HTTP POST.  
We send a `message/send` request with a text message, and receive a completed **Task** with the agent's response.

In [ ]:
import requests, json, uuid

utils.print_info("Calling Title Agent → message/send(topic='Artificial Intelligence')...")
request = {
    "jsonrpc": "2.0",
    "method": "message/send",
    "id": str(uuid.uuid4()),
    "params": {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": "Artificial Intelligence"}],
            "messageId": str(uuid.uuid4())
        }
    }
}

response = requests.post(title_agent_endpoint,
                         headers={"Content-Type": "application/json"},
                         json=request)

if response.status_code == 200:
    result = response.json()
    utils.print_ok("Title Agent: HTTP 200 ✅")
    
    # Parse JSON-RPC response
    task = result.get('result', {})
    status = task.get('status', {})
    state = status.get('state')
    message = status.get('message', {})
    parts = message.get('parts', [])
    
    print(f"  Task ID:     {task.get('id')}")
    print(f"  State:       {state}")
    if parts:
        print(f"  Response:    {parts[0].get('text')}")
    
    if state == 'completed':
        utils.print_ok("Task completed successfully ✅")
    else:
        utils.print_error(f"Unexpected state: {state}")
else:
    utils.print_error(f"Title Agent: HTTP {response.status_code} - {response.text[:200]}")

### 🧪 Test Outline Agent — A2A `message/send`

Same protocol, different agent. The Outline Agent generates a structured outline for a given topic.

In [ ]:
import requests, json, uuid

utils.print_info("Calling Outline Agent → message/send(topic='Cloud Computing')...")
request = {
    "jsonrpc": "2.0",
    "method": "message/send",
    "id": str(uuid.uuid4()),
    "params": {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": "Cloud Computing"}],
            "messageId": str(uuid.uuid4())
        }
    }
}

response = requests.post(outline_agent_endpoint,
                         headers={"Content-Type": "application/json"},
                         json=request)

if response.status_code == 200:
    result = response.json()
    utils.print_ok("Outline Agent: HTTP 200 ✅")
    
    task = result.get('result', {})
    status = task.get('status', {})
    state = status.get('state')
    message = status.get('message', {})
    parts = message.get('parts', [])
    
    print(f"  Task ID:     {task.get('id')}")
    print(f"  State:       {state}")
    if parts:
        outline_text = parts[0].get('text', '')
        print(f"  Outline:")
        for line in outline_text.split('\n'):
            print(f"    {line}")
    
    if state == 'completed':
        utils.print_ok("Task completed successfully ✅")
    else:
        utils.print_error(f"Unexpected state: {state}")
else:
    utils.print_error(f"Outline Agent: HTTP {response.status_code} - {response.text[:200]}")

### 🧪 Test Error Handling — Unknown JSON-RPC Method

A2A agents should return a JSON-RPC error for unsupported methods (code `-32601`).

In [ ]:
import requests, json, uuid

utils.print_info("Calling Title Agent with unsupported method → expect error -32601...")
request = {
    "jsonrpc": "2.0",
    "method": "tasks/cancel",
    "id": str(uuid.uuid4()),
    "params": {"taskId": "nonexistent"}
}

response = requests.post(title_agent_endpoint,
                         headers={"Content-Type": "application/json"},
                         json=request)

if response.status_code == 200:
    result = response.json()
    error = result.get('error', {})
    if error.get('code') == -32601:
        utils.print_ok(f"JSON-RPC error returned correctly: {error.get('code')} — {error.get('message')} ✅")
    else:
        utils.print_error(f"Unexpected response: {json.dumps(result, indent=2)}")
else:
    utils.print_error(f"HTTP {response.status_code}")

---
## Part 4 — Add a New A2A Agent and Show Auto-Discovery
---

> 💡 **This is the key demo moment.**  
> We deploy a **3rd A2A agent** (Summary Generator) into the same APIM + API Center.  
> After deployment, it **automatically appears** in API Center — no manual registration needed.

### 4️⃣ Deploy Summary Agent as an add-on

This uses a separate Bicep file that targets the **existing** APIM and API Center resources.  
It deploys the Summary Generator REST API + its A2A server.

In [ ]:
summary_deployment_name = f"{deployment_name}-summary"

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimServiceName": { "value": apim_service_name },
        "apicServiceName": { "value": apic_service_name },
        "apicApiEnvironmentName": { "value": apic_api_env },
        "apicA2aEnvironmentName": { "value": apic_a2a_env }
    }
}

with open('params-a2a-demo-summary.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {summary_deployment_name} --resource-group {resource_group_name} --template-file demo-a2a-add-summary.bicep --parameters params-a2a-demo-summary.json",
    f"✅ Summary Agent deployed!",
    f"❌ Summary Agent deployment failed"
)

if output.success and output.json_data:
    summary_agent_endpoint = output.json_data['properties']['outputs']['summaryAgentEndpoint']['value']
    summary_agent_card_endpoint = output.json_data['properties']['outputs']['summaryAgentCardEndpoint']['value']
    utils.print_info(f"Summary Agent Endpoint: {summary_agent_endpoint}")
    utils.print_info(f"Summary Agent Card:     {summary_agent_card_endpoint}")

### 5️⃣ 🔍 Verify auto-discovery — Summary Agent now appears in API Center!

Compare this output with the earlier API Center listing.  
You will see **2 new entries**: `summary-generator-api` (rest) and `summary-agent` (a2a).

In [ ]:
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Count and snapshot
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} -o json',
    "", "")
if output.success and output.json_data:
    apis_after = output.json_data
    rest_after = sum(1 for api in apis_after if api.get('kind') == 'rest')
    a2a_after = sum(1 for api in apis_after if api.get('kind') == 'a2a')
    print(f"\n📊 Total: {len(apis_after)} APIs registered — {rest_after} REST APIs, {a2a_after} A2A Agents")
    print(f"🆕 Summary Agent is now discoverable!")

### ✅ Validate: Before vs After — Auto-discovery proof

Compare the API Center snapshots taken before and after adding the Summary Agent.

In [ ]:
# Before vs After comparison
names_before = {api['name'] for api in apis_before}
names_after  = {api['name'] for api in apis_after}
new_apis = names_after - names_before

print("🔍 Validation — Before vs After Comparison")
print("=" * 55)

rest_before = sum(1 for api in apis_before if api.get('kind') == 'rest')
a2a_before  = sum(1 for api in apis_before if api.get('kind') == 'a2a')
rest_after  = sum(1 for api in apis_after if api.get('kind') == 'rest')
a2a_after   = sum(1 for api in apis_after if api.get('kind') == 'a2a')

print(f"  {'':20} {'BEFORE':>10} {'AFTER':>10} {'DIFF':>10}")
print(f"  {'-'*20} {'-'*10} {'-'*10} {'-'*10}")
print(f"  {'REST APIs':20} {rest_before:>10} {rest_after:>10} {'+' + str(rest_after - rest_before):>10}")
print(f"  {'A2A Agents':20} {a2a_before:>10} {a2a_after:>10} {'+' + str(a2a_after - a2a_before):>10}")
print(f"  {'Total':20} {len(apis_before):>10} {len(apis_after):>10} {'+' + str(len(apis_after) - len(apis_before)):>10}")
print()

if new_apis:
    print(f"  🆕 Newly discovered APIs:")
    for name in sorted(new_apis):
        kind = next((api['kind'] for api in apis_after if api['name'] == name), '?')
        print(f"     • {name} ({kind})")
    print()

# Final assertions
all_passed = True
if "summary-generator-api" in names_after and "summary-agent" in names_after:
    utils.print_ok("Summary API + A2A Agent auto-discovered in API Center ✅")
else:
    utils.print_error("Summary Agent not found in API Center ❌")
    all_passed = False

if rest_after == rest_before + 1:
    utils.print_ok(f"REST API count increased by 1 ({rest_before} → {rest_after}) ✅")
else:
    utils.print_error(f"Unexpected REST count: {rest_before} → {rest_after}")
    all_passed = False

if a2a_after == a2a_before + 1:
    utils.print_ok(f"A2A Agent count increased by 1 ({a2a_before} → {a2a_after}) ✅")
else:
    utils.print_error(f"Unexpected A2A count: {a2a_before} → {a2a_after}")
    all_passed = False

if all_passed:
    print()
    utils.print_ok("🎉 Auto-discovery validated — new A2A agents appear automatically!")

---
## Part 5 — Test Summary Agent
---

### 🔍 Discover Summary Agent Card

In [ ]:
import requests, json

utils.print_info(f"Discovering Summary Agent → GET {summary_agent_card_endpoint}")
response = requests.get(summary_agent_card_endpoint, headers={"Accept": "application/json"})

if response.status_code == 200:
    card = response.json()
    utils.print_ok("Summary Agent: Agent Card retrieved ✅")
    print(f"  Name:        {card.get('name')}")
    print(f"  Version:     {card.get('version')}")
    print(f"  URL:         {card.get('url')}")
    skills = card.get('skills', [])
    for skill in skills:
        print(f"  Skill:       {skill.get('id')} — {skill.get('description')}")
else:
    utils.print_error(f"Summary Agent: HTTP {response.status_code}")

### 🧪 Test Summary Agent — A2A `message/send`

Send a text to the Summary Agent and get a concise summary back.

In [ ]:
import requests, json, uuid

sample_text = (
    "Cloud computing has revolutionized the way businesses operate. "
    "It provides scalable resources on demand. Companies no longer need to invest heavily in physical hardware. "
    "Instead, they can leverage cloud providers for compute, storage, and networking. "
    "This shift has enabled startups to compete with established enterprises. "
    "The pay-as-you-go model reduces upfront costs significantly. "
    "Security in the cloud has also matured considerably over the years. "
    "Major providers now offer comprehensive compliance certifications. "
    "Multi-cloud strategies are becoming increasingly popular among large organizations. "
    "Edge computing is emerging as a complementary technology to traditional cloud services."
)

utils.print_info("Calling Summary Agent → message/send with sample text...")
request = {
    "jsonrpc": "2.0",
    "method": "message/send",
    "id": str(uuid.uuid4()),
    "params": {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": sample_text}],
            "messageId": str(uuid.uuid4())
        }
    }
}

response = requests.post(summary_agent_endpoint,
                         headers={"Content-Type": "application/json"},
                         json=request)

if response.status_code == 200:
    result = response.json()
    utils.print_ok("Summary Agent: HTTP 200 ✅")
    
    task = result.get('result', {})
    status = task.get('status', {})
    state = status.get('state')
    message = status.get('message', {})
    parts = message.get('parts', [])
    
    print(f"  Task ID:     {task.get('id')}")
    print(f"  State:       {state}")
    if parts:
        print(f"  Summary:     {parts[0].get('text')}")
    
    print(f"\n  Original text length: {len(sample_text.split())} words")
    if parts:
        summary_text = parts[0].get('text', '')
        print(f"  Summary length:       {len(summary_text.split())} words")
    
    if state == 'completed':
        utils.print_ok("Task completed successfully ✅")
    else:
        utils.print_error(f"Unexpected state: {state}")
else:
    utils.print_error(f"Summary Agent: HTTP {response.status_code} - {response.text[:200]}")

---
## ✅ Demo Summary
---

In [ ]:
print("=" * 68)
print("  🤝  A2A AGENT DEMO SUMMARY")
print("=" * 68)
print(f"  Resource Group:  {resource_group_name}")
print(f"  APIM Gateway:    {apim_resource_gateway_url}")
print(f"  API Center:      {apic_service_name}")
print()
print("  A2A Agents Deployed:")
print(f"    1. Title Agent:        {title_agent_endpoint}")
print(f"       Agent Card:         {title_agent_card_endpoint}")
print(f"    2. Outline Agent:      {outline_agent_endpoint}")
print(f"       Agent Card:         {outline_agent_card_endpoint}")
print(f"    3. Summary Agent:      {summary_agent_endpoint}  🆕")
print(f"       Agent Card:         {summary_agent_card_endpoint}  🆕")
print()
print("  Key takeaways:")
print("    • REST APIs transformed to A2A agents via APIM policies")
print("    • Agent Cards served at /.well-known/agent.json")
print("    • JSON-RPC 2.0 message/send protocol fully handled")
print("    • New A2A agents auto-discovered in API Center (kind: a2a)")
print("    • Full observability via App Insights tracing")
print("=" * 68)

---
## 🗑️ Clean up resources

Uncomment and run the cell below to delete all demo resources.

In [ ]:
# Uncomment to delete all demo resources:
# utils.run(f"az group delete --name {resource_group_name} --yes --no-wait",
#           f"Resource group '{resource_group_name}' deletion initiated",
#           f"Failed to delete resource group")